In [85]:
import sys
from collections import defaultdict
from itertools import *
import random
import numpy as np
import pdb
from scipy.sparse import *
from scipy.sparse.linalg import svds
import time

from pyspark import SparkContext
sc=SparkContext.getOrCreate()

In [86]:
#TRAINSET LOAD
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))

In [87]:
mu= trainSet.map(lambda x: x[2]).mean()
print("mu: ", mu)

mu:  6.8188763965700625


In [88]:
#remove mu from each element
trainSet = trainSet.map(lambda x: (x[0], x[1], x[2] - mu))
trainSet.take(5)

[(2738, 1, -5.8188763965700625),
 (4716, 1, -2.8188763965700625),
 (13298, 1, 1.1811236034299375),
 (15122, 1, -2.8188763965700625),
 (11326, 2, -1.8188763965700625)]

In [89]:
#Compute bi, item bias
reg1=20
itemBias = trainSet.map(lambda x: (x[1],x[2])).groupByKey().map(lambda x: (x[0], sum(x[1].data)/(len(x[1])+reg1)))
b_i_dict=itemBias.collectAsMap()

In [90]:
#Compute bi, item bias
reg2=7
userBias = trainSet.map(lambda x: (x[0],x[2] - b_i_dict[x[1]])).groupByKey().map(lambda x: (x[0], sum(x[1].data)/(len(x[1])+reg2)))
b_u_dict=userBias.collectAsMap()

In [91]:
trainSet = trainSet.map(lambda x: (x[0], x[1], x[2] + mu))
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
userItems = csr_matrix((data, (rows, cols)))
userItems.shape

(15375, 37143)

In [92]:
u, s, v = svds(userItems, k=300, which="LM")
v=v.transpose()

In [93]:
targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

In [94]:
#example of prediction:
#predict for user 100

s=time.time()
items = b_i_dict.keys()

pred100 = []
for i in items:
    pred100.append((i,mu + b_i_dict[i] + b_u_dict[100] + np.dot(u[100],v[i])))
pred=sorted(pred100, key = lambda x: -x[1])
print("Exec time: ",time.time() - s)
pred[:5]


Exec time:  0.11408209800720215


[(33475, 9.0411771339082456),
 (33173, 8.9577850143637452),
 (15743, 8.710092153992278),
 (32347, 8.7082702020521268),
 (36872, 8.694341285764807)]

In [95]:
numberOfRecommendations = 5
finalPred = []
k = 0
for target in targets:
    k=k+1
    pred = []
    for i in items:
        pred.append((i, mu + b_i_dict[i] + b_u_dict[target] + np.dot(u[target],v[i])))
    recommendations = sorted(pred, key = lambda x: -x[1])[:numberOfRecommendations]
    finalPred.append((target, recommendations))
    print(k)
    
finalPred

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


[(4,
  [(33475, 9.4652954199682355),
   (33173, 9.3841494196165502),
   (15743, 9.1363744532950477),
   (32347, 9.1326193714498256),
   (36872, 9.1203079895586487)]),
 (5,
  [(33475, 8.2734993388203275),
   (33173, 8.1896185026236044),
   (15743, 7.9491936033716941),
   (32347, 7.9372526488105288),
   (36872, 7.9260511757570979)]),
 (8,
  [(33475, 9.7744689125894784),
   (33173, 9.6929147270305549),
   (15743, 9.4468786160928229),
   (32347, 9.4403055317330242),
   (36872, 9.4302035318010518)]),
 (9,
  [(33475, 9.4560180214842529),
   (33173, 9.3703045049401705),
   (15743, 9.1241256638569581),
   (32347, 9.1176971888445841),
   (36872, 9.1077094525186055)]),
 (13,
  [(33475, 9.087512026657393),
   (33173, 9.002867384689706),
   (15743, 8.7562764831677509),
   (32347, 8.7512018290601468),
   (36872, 8.7412774550276762)]),
 (18,
  [(33475, 9.2551940316707686),
   (33173, 9.1613852704781209),
   (15743, 8.9163093741926804),
   (32347, 8.9120853048592092),
   (36872, 8.9029435666157557)])

In [96]:
f=open("predictionSVDUB.csv",'w')
i=0
f.write("userId,RecommendedItemIds\n")
for (user, (prods)) in finalPred:
    f.write(str(user)+',')
    for (item,rating) in prods:
        f.write(str(item)+' ')
    f.write('\n')
    i=i+1
    print(i,"of", len(finalPred), "written")
f.close()

1 of 4196 written
2 of 4196 written
3 of 4196 written
4 of 4196 written
5 of 4196 written
6 of 4196 written
7 of 4196 written
8 of 4196 written
9 of 4196 written
10 of 4196 written
11 of 4196 written
12 of 4196 written
13 of 4196 written
14 of 4196 written
15 of 4196 written
16 of 4196 written
17 of 4196 written
18 of 4196 written
19 of 4196 written
20 of 4196 written
21 of 4196 written
22 of 4196 written
23 of 4196 written
24 of 4196 written
25 of 4196 written
26 of 4196 written
27 of 4196 written
28 of 4196 written
29 of 4196 written
30 of 4196 written
31 of 4196 written
32 of 4196 written
33 of 4196 written
34 of 4196 written
35 of 4196 written
36 of 4196 written
37 of 4196 written
38 of 4196 written
39 of 4196 written
40 of 4196 written
41 of 4196 written
42 of 4196 written
43 of 4196 written
44 of 4196 written
45 of 4196 written
46 of 4196 written
47 of 4196 written
48 of 4196 written
49 of 4196 written
50 of 4196 written
51 of 4196 written
52 of 4196 written
53 of 4196 written
54

767 of 4196 written
768 of 4196 written
769 of 4196 written
770 of 4196 written
771 of 4196 written
772 of 4196 written
773 of 4196 written
774 of 4196 written
775 of 4196 written
776 of 4196 written
777 of 4196 written
778 of 4196 written
779 of 4196 written
780 of 4196 written
781 of 4196 written
782 of 4196 written
783 of 4196 written
784 of 4196 written
785 of 4196 written
786 of 4196 written
787 of 4196 written
788 of 4196 written
789 of 4196 written
790 of 4196 written
791 of 4196 written
792 of 4196 written
793 of 4196 written
794 of 4196 written
795 of 4196 written
796 of 4196 written
797 of 4196 written
798 of 4196 written
799 of 4196 written
800 of 4196 written
801 of 4196 written
802 of 4196 written
803 of 4196 written
804 of 4196 written
805 of 4196 written
806 of 4196 written
807 of 4196 written
808 of 4196 written
809 of 4196 written
810 of 4196 written
811 of 4196 written
812 of 4196 written
813 of 4196 written
814 of 4196 written
815 of 4196 written
816 of 4196 written


1312 of 4196 written
1313 of 4196 written
1314 of 4196 written
1315 of 4196 written
1316 of 4196 written
1317 of 4196 written
1318 of 4196 written
1319 of 4196 written
1320 of 4196 written
1321 of 4196 written
1322 of 4196 written
1323 of 4196 written
1324 of 4196 written
1325 of 4196 written
1326 of 4196 written
1327 of 4196 written
1328 of 4196 written
1329 of 4196 written
1330 of 4196 written
1331 of 4196 written
1332 of 4196 written
1333 of 4196 written
1334 of 4196 written
1335 of 4196 written
1336 of 4196 written
1337 of 4196 written
1338 of 4196 written
1339 of 4196 written
1340 of 4196 written
1341 of 4196 written
1342 of 4196 written
1343 of 4196 written
1344 of 4196 written
1345 of 4196 written
1346 of 4196 written
1347 of 4196 written
1348 of 4196 written
1349 of 4196 written
1350 of 4196 written
1351 of 4196 written
1352 of 4196 written
1353 of 4196 written
1354 of 4196 written
1355 of 4196 written
1356 of 4196 written
1357 of 4196 written
1358 of 4196 written
1359 of 4196 

1801 of 4196 written
1802 of 4196 written
1803 of 4196 written
1804 of 4196 written
1805 of 4196 written
1806 of 4196 written
1807 of 4196 written
1808 of 4196 written
1809 of 4196 written
1810 of 4196 written
1811 of 4196 written
1812 of 4196 written
1813 of 4196 written
1814 of 4196 written
1815 of 4196 written
1816 of 4196 written
1817 of 4196 written
1818 of 4196 written
1819 of 4196 written
1820 of 4196 written
1821 of 4196 written
1822 of 4196 written
1823 of 4196 written
1824 of 4196 written
1825 of 4196 written
1826 of 4196 written
1827 of 4196 written
1828 of 4196 written
1829 of 4196 written
1830 of 4196 written
1831 of 4196 written
1832 of 4196 written
1833 of 4196 written
1834 of 4196 written
1835 of 4196 written
1836 of 4196 written
1837 of 4196 written
1838 of 4196 written
1839 of 4196 written
1840 of 4196 written
1841 of 4196 written
1842 of 4196 written
1843 of 4196 written
1844 of 4196 written
1845 of 4196 written
1846 of 4196 written
1847 of 4196 written
1848 of 4196 

2326 of 4196 written
2327 of 4196 written
2328 of 4196 written
2329 of 4196 written
2330 of 4196 written
2331 of 4196 written
2332 of 4196 written
2333 of 4196 written
2334 of 4196 written
2335 of 4196 written
2336 of 4196 written
2337 of 4196 written
2338 of 4196 written
2339 of 4196 written
2340 of 4196 written
2341 of 4196 written
2342 of 4196 written
2343 of 4196 written
2344 of 4196 written
2345 of 4196 written
2346 of 4196 written
2347 of 4196 written
2348 of 4196 written
2349 of 4196 written
2350 of 4196 written
2351 of 4196 written
2352 of 4196 written
2353 of 4196 written
2354 of 4196 written
2355 of 4196 written
2356 of 4196 written
2357 of 4196 written
2358 of 4196 written
2359 of 4196 written
2360 of 4196 written
2361 of 4196 written
2362 of 4196 written
2363 of 4196 written
2364 of 4196 written
2365 of 4196 written
2366 of 4196 written
2367 of 4196 written
2368 of 4196 written
2369 of 4196 written
2370 of 4196 written
2371 of 4196 written
2372 of 4196 written
2373 of 4196 

2820 of 4196 written
2821 of 4196 written
2822 of 4196 written
2823 of 4196 written
2824 of 4196 written
2825 of 4196 written
2826 of 4196 written
2827 of 4196 written
2828 of 4196 written
2829 of 4196 written
2830 of 4196 written
2831 of 4196 written
2832 of 4196 written
2833 of 4196 written
2834 of 4196 written
2835 of 4196 written
2836 of 4196 written
2837 of 4196 written
2838 of 4196 written
2839 of 4196 written
2840 of 4196 written
2841 of 4196 written
2842 of 4196 written
2843 of 4196 written
2844 of 4196 written
2845 of 4196 written
2846 of 4196 written
2847 of 4196 written
2848 of 4196 written
2849 of 4196 written
2850 of 4196 written
2851 of 4196 written
2852 of 4196 written
2853 of 4196 written
2854 of 4196 written
2855 of 4196 written
2856 of 4196 written
2857 of 4196 written
2858 of 4196 written
2859 of 4196 written
2860 of 4196 written
2861 of 4196 written
2862 of 4196 written
2863 of 4196 written
2864 of 4196 written
2865 of 4196 written
2866 of 4196 written
2867 of 4196 

3314 of 4196 written
3315 of 4196 written
3316 of 4196 written
3317 of 4196 written
3318 of 4196 written
3319 of 4196 written
3320 of 4196 written
3321 of 4196 written
3322 of 4196 written
3323 of 4196 written
3324 of 4196 written
3325 of 4196 written
3326 of 4196 written
3327 of 4196 written
3328 of 4196 written
3329 of 4196 written
3330 of 4196 written
3331 of 4196 written
3332 of 4196 written
3333 of 4196 written
3334 of 4196 written
3335 of 4196 written
3336 of 4196 written
3337 of 4196 written
3338 of 4196 written
3339 of 4196 written
3340 of 4196 written
3341 of 4196 written
3342 of 4196 written
3343 of 4196 written
3344 of 4196 written
3345 of 4196 written
3346 of 4196 written
3347 of 4196 written
3348 of 4196 written
3349 of 4196 written
3350 of 4196 written
3351 of 4196 written
3352 of 4196 written
3353 of 4196 written
3354 of 4196 written
3355 of 4196 written
3356 of 4196 written
3357 of 4196 written
3358 of 4196 written
3359 of 4196 written
3360 of 4196 written
3361 of 4196 

3812 of 4196 written
3813 of 4196 written
3814 of 4196 written
3815 of 4196 written
3816 of 4196 written
3817 of 4196 written
3818 of 4196 written
3819 of 4196 written
3820 of 4196 written
3821 of 4196 written
3822 of 4196 written
3823 of 4196 written
3824 of 4196 written
3825 of 4196 written
3826 of 4196 written
3827 of 4196 written
3828 of 4196 written
3829 of 4196 written
3830 of 4196 written
3831 of 4196 written
3832 of 4196 written
3833 of 4196 written
3834 of 4196 written
3835 of 4196 written
3836 of 4196 written
3837 of 4196 written
3838 of 4196 written
3839 of 4196 written
3840 of 4196 written
3841 of 4196 written
3842 of 4196 written
3843 of 4196 written
3844 of 4196 written
3845 of 4196 written
3846 of 4196 written
3847 of 4196 written
3848 of 4196 written
3849 of 4196 written
3850 of 4196 written
3851 of 4196 written
3852 of 4196 written
3853 of 4196 written
3854 of 4196 written
3855 of 4196 written
3856 of 4196 written
3857 of 4196 written
3858 of 4196 written
3859 of 4196 

In [69]:
#print(v[33475])
#print(u[3602])
print("b_i:", b_i_dict[33475])
print("b_u:", b_u_dict[3602])
print(np.dot(v[33475],u[3602]))

b_i: 2.638011243086944
b_u: -0.30274564273704907
-0.0127601983764


In [78]:
i33475 = trainSet.map(lambda x: (x[1],x[2] - mu)).groupByKey().filter(lambda x: x[0]==33475)


(33475, <pyspark.resultiterable.ResultIterable at 0x261989a2e80>)

In [84]:
voti=i33475.collect()[0][1].data
somma = sum(voti)
print("Somma: ",somma)
print("Lunghezza: ", len(voti))
i_b=somma/(len(voti)+20)
print(i_b)

Somma:  211.04089944695554
Lunghezza:  72
2.2939228200756037
